<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/d3_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using D3 In Jupyter Notebooks On Google Colab

**Modules used to dislay HTML in Jupyter Notebook and convert between Python dictionaries and json files.**

In [1]:
from IPython.display import display, HTML
import json

In [2]:
!node -v

v14.16.0


In [3]:
# !wget "https://d3js.org/d3.v6.min.js"
# !ls

**Using the Node Requires access to d3 does not seem to work on Google Colab.**

In [21]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min.js"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

**Using the `dislay()` function we can show different data types with fancy formating in our return cell.**

**Using `HTML()` we can pass HTML to our display.** 

In [5]:
display(HTML("""
<html>
  <body>
    <div style="width:50px; height: 50px; background-color:#CCFF00"/>
  </body>
</html>
"""))

**We can store our javascript as a string and/or write our javascript to a file.**

In [95]:
library = """
function circle(){
    d3.select("#div_id")
    .append("div")
    .style("width", "50px")
    .style("height", "50px")
    .style("background-color", "#00CCCC")
    .style("border-radius", "50px")
}

function bar_chart(div_id, data){
    const div = d3.select(div_id)
    const height = data.length * 20;
    
    const scaleY = d3.scaleBand()
        .domain(d3.range(data.length))
        .range([0, height])
        .paddingInner(0.1);
    const scaleX = d3.scaleLinear()
        .domain(d3.extent(data))
        .range([1, 120]);
    
    const svg = div.append("svg")
        .style("width", "120px")
        .style("height", height+"px");
    
    svg.selectAll("rect")
        .data(data)
        .enter()
        .append("rect")
        .attr("x", 0)
        .attr("y", (d, i) => scaleY(i))
        .attr("width", d => scaleX(d))
        .attr("height", scaleY.bandwidth);
}
"""

with open('library.js', 'w') as f:
  f.write(library)
  f.close()

**Check the file has been written.**

In [7]:
!ls

library.js  sample_data


**Read the text from the file.**

In [8]:
!cat library.js


function circle(){
    d3.select("#div_id")
    .append("div")
    .style("width", "50px")
    .style("height", "50px")
    .style("background-color", "#00CCCC")
    .style("border-radius", "50px")
}

function bar_chart(div_id, data){
    const div = d3.select(div_id)
    const height = data.length * 20;
    
    const scaleY = d3.scaleBand()
        .domain(d3.range(data.length))
        .range([0, height])
        .paddingInner(0.1);
    const scaleX = d3.scaleLinear()
        .domain(d3.extent(data))
        .range([1, 120]);
    
    const svg = div.append("svg")
        .style("width", "120px")
        .style("height", height+"px");
    
    svg.selectAll("rect")
        .data(data)
        .enter()
        .append("rect")
        .attr("x", 0)
        .attr("y", (d, i) => scaleY(i))
        .attr("width", d => scaleX(d))
        .attr("height", scaleY.bandwidth);
}


**Display d3 visualization using js from our file.**

In [24]:
with open('library.js', 'r') as f:
    lib = f.read()
    f.close()
my_html = """
<html>
    <head>
        <script src="https://requirejs.org/docs/release/2.3.6/minified/require.js"></script>
        //<script src="https://d3js.org/d3.v6.min.js"></script>
        <script>{library}</script>
    </head>
    <body>
        <div id="div_id"/>
    </body>
    <script>
        circle();
    </script>
</html>
""".format(library=lib)

display(HTML(my_html))
# code is inserted within string 
# so full page is being passed to HTML function as a string

**Display d3 visualization using js from our string variable in python.**

In [12]:
my_html = """
<html>
    <head>
        <script src="https://d3js.org/d3.v6.min.js"></script>
        <script>{library}</script>
    </head>
    <body>
        <div id="div_id"/>
    </body>
    <script>
        circle();
    </script>
</html>
""".format(library=library)

display(HTML(my_html))
# code is inserted within string 
# so full page is being passed to HTML function as a string

**We can use `json.dumps()` to convert a python list to a string used by javascript as an array.**

In [13]:
json.dumps([-3, -1, -2, 1, 2, 3, 4])

'[-3, -1, -2, 1, 2, 3, 4]'

**Put the data string into our html string where the custom d3 bar graph function is called with our data and display the results as HTML.**

In [10]:
html_str = """
<html>
<head>
<script src="https://d3js.org/d3.v6.min.js"></script>
</head>
<body>
<div id="div_data1"/>
</body>
<script type="text/javascript">
{library}
bar_chart("#div_data1", {data})
</script>
</html>
""".format(library=library, data=json.dumps([-3, -1, -2, 1, 2, 3, 4]))
display(HTML(html_str))

**We can also use `json.dumps()` to convert a python dictionary to as json string that can be used by javascript.**

In [14]:
python_dict = {'data':[-3, -1, -2, 1, 2, 3, 4]}
json.dumps(python_dict)

'{"data": [-3, -1, -2, 1, 2, 3, 4]}'

**Sending data from JS to Python**

In [85]:
import random
def target_func(comm, open_msg):
    # comm is the kernel Comm instance

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        # Use msg['content']['data'] for the data in the message
        n = msg['content']['data']['n']
        comm.send({'array': [random.random() for x in range(n)]})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [88]:
from string import Template

with open('library.js', 'r') as f:
    lib = f.read()

template = Template("""
<html>
<head>
<script src="https://d3js.org/d3.v6.min.js"></script>
</head>
<body>
<label># Bars: <input type="text" id="text_n" value="3"/></label> <input type="button" value="Send" id="button"/>
<div id="div_receive_data"/>
</body>
<script>
{library}

# function main(){
#     const channel = await google.colab.kernel.comms.open('my_comm_target','data', {'n':3})
#     // Send data
#     comm.send({'n': 3});

#     // Register a handler
#     comm.on_msg(function(msg) {
#         let data = msg.content.data.array;
#         d3.select("#div_receive_data").selectAll("*").remove()
#         bar_chart("#div_receive_data", data)
#     });
    
#     // Setting up button
#     document.getElementById("button").addEventListener("click", ()=>{
#         let n = +document.getElementById("text_n").value;
#         comm.send({'n': n});
#     }); 
#     bar_chart("#div_receive_data", {'n':3});
# }
# main();
bar_chart("#div_receive_data", {'n':3});
</script>
</html>
""")
my_html = template.safe_substitute(library=library)
display(HTML(my_html))

In [89]:
import ipywidgets as widgets

slider = widgets.IntSlider(20, min=0, max=100)
slider

@widgets.interact(x=slider)
def render(x):
  print(x)

interactive(children=(IntSlider(value=20, description='x'), Output()), _dom_classes=('widget-interact',))

In [98]:
import ipywidgets as widgets

slider = widgets.IntSlider(5, min=0, max=10)
slider

@widgets.interact(x=slider)
def render(x):
  print(x)
  html_str = """
  <html>
  <head>
  <script src="https://d3js.org/d3.v6.min.js"></script>
  </head>
  <body>
  <div id="div_data1"/>
  </body>
  <script type="text/javascript">
  {library}
  bar_chart("#div_data1", {data})
  </script>
  </html>
  """.format(library=library, data=json.dumps([random.random() for _ in range(x)]))
  display(HTML(html_str))

interactive(children=(IntSlider(value=5, description='x', max=10), Output()), _dom_classes=('widget-interact',…

In [81]:
from IPython.display import Javascript

# Function to receive data from javascript
def target_func(comm, msg):
  print(msg['content']['data'])

# Register python function to be called from Javascript
get_ipython().kernel.comm_manager.register_target('comm_target', target_func)

# Call javascript to run
Javascript('''
(async () => {
  const channel = await google.colab.kernel.comms.open('comm_target', '[0,1,2]');
  document.body.appendChild(document.createTextNode('done.'));
})()
''')

<IPython.core.display.Javascript object>

[0,1,2]


## display.Javascript to execute JavaScript from Python

The Javascript is persisted in the outputs of the notebook document and will be executed when the document is reloaded.

In [25]:
import IPython
js_code = '''
document.querySelector("#output-area").appendChild(document.createTextNode("hello world!"));
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

## Evaluate a Javascript expression from Python with eval_js

Evaluates the Javascript expression within the context of the outputframe of the current cell.

This executes the Javascript as a blocking call from Python and returns the result of the expression.

If the Javascript expression results in a [Promise](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Promise) then the call will block until the promise has settled and use the resulting value.

**Note:** This differs from `display.Javascript` in that the Javascript expression is not persisted in the notebook document and will not be executed when the document is reloaded.

In [26]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  window.someValue = new Promise(resolve => {
    setTimeout(() => {
      resolve("hello world!");
    }, 100);
  });
'''))


value = output.eval_js('someValue');
value

<IPython.core.display.Javascript object>

'hello world!'

## Javascript to Python communication

Python code can register callbacks which can be invoked by Javascript in the outputframe.

**Note:** This is only available to 'trusted' outputs- outputs which have been executed within the current session.

The Javascript APIs are documented in Colab's [outputframe type declarations](https://github.com/googlecolab/colabtools/blob/master/packages/outputframe/lib/index.d.ts).

In [31]:
import IPython
from google.colab import output

def Concat(a, b):
  # Use display.JSON to transfer a structured result.
  return IPython.display.JSON({'result': ' '.join((a, b))})

output.register_callback('notebook.Concat', Concat)

In [35]:
%%javascript
(async function() {
  const result = await google.colab.kernel.invokeFunction(
    'notebook.Concat', // The callback name.
    ['hello', 'world!'], // The arguments.
    {}); // kwargs
  console.log(result);
  const text = result.data['application/json'];
  console.log(text);
  document.querySelector("#output-area").appendChild(document.createTextNode(text.result));
  console.log(document);
})();

<IPython.core.display.Javascript object>

### Persisting Updates

Since `invokeFunction` is only available for outputs which have executed within the current session
then it may be desirable to update the notebook so the changes take effect on notebook reload. An example
of doing this.

In [36]:
import IPython
from google.colab import output

display(IPython.display.HTML('''
    The items:
    <br><ol id="items"></ol>
    <button id='button'>Click to add</button>
    <script>
      document.querySelector('#button').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.AddListItem', [], {});
      };
    </script>
    '''))

def add_list_item():
  # Use redirect_to_element to direct the elements which are being written.
  with output.redirect_to_element('#items'):
    # Use display to add items which will be persisted on notebook reload.
    display(IPython.display.HTML('<li> Another item</li>'))

output.register_callback('notebook.AddListItem', add_list_item)

In [37]:
import IPython
import uuid
from google.colab import output

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def do_something():
  print('here')

InvokeButton('click me', do_something)

## Jupyter Widgets

[Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html) can be used to add interactivity to notebooks.

In [40]:
import ipywidgets as widgets

slider = widgets.IntSlider(20, min=0, max=100)
slider

IntSlider(value=20)

### Use a Select widget to pick the lines for a chart.

In [42]:
import altair as alt
import ipywidgets as widgets
from vega_datasets import data

# Get data
source = data.stocks()

# Create widget
stock_picker = widgets.SelectMultiple(
    options=source.symbol.unique(),
    value=list(source.symbol.unique()),
    description='Symbols')

# decorator runs function with the current widget input
@widgets.interact(symbols=stock_picker)
def render(symbols):
  selected = source[source.symbol.isin(list(symbols))]

  return alt.Chart(selected).mark_line().encode(
      x='date',
      y='price',
      color='symbol',
      strokeDash='symbol',
  )

interactive(children=(SelectMultiple(description='Symbols', index=(0, 1, 2, 3, 4), options=('MSFT', 'AMZN', 'I…

## Jupyter Comms

[Jupyter Comms](https://jupyter-notebook.readthedocs.io/en/stable/comms.html) can be used for bidirectional communication between the kernel and notebook.

The Javascript APIs are documented in Colab's [outputframe type declarations](https://github.com/googlecolab/colabtools/blob/master/packages/outputframe/lib/index.d.ts).



### Establish a comm channel from client to kernel

This registers a comm target on the kernel then when the output is displayed in the client it establishes a comm channel from the client to the kernel.

In [ ]:
from IPython.display import Javascript

def target_func(comm, msg):
  # Only send the response if it's the data we are expecting.
  if msg['content']['data'] == 'the data':
    comm.send({
          'response': 'got comm open!',
        }, None, msg['buffers']);
get_ipython().kernel.comm_manager.register_target('comm_target', target_func)



In [53]:
from IPython.display import Javascript

# Make a python function
# Register the function as a target
def target_func(comm, msg):
  # Only send the response if it's the data we are expecting.
  print(comm)
  print(msg)
  if msg['content']['data'] == 'the data':
    comm.send({
          'response': 'got comm open!',
        }, None, msg['buffers']);
get_ipython().kernel.comm_manager.register_target('comm_target', target_func)

Javascript('''
(async () => {
  const buffer = new Uint8Array(10);
  console.log(buffer);
  for (let i = 0; i < buffer.byteLength; ++i) {
    buffer[i] = i
  }
  console.log(buffer);
  const channel = await google.colab.kernel.comms.open('comm_target', 'the data', [buffer.buffer]);
  console.log(channel);
  let success = false;
  console.log(success);
  for await (const message of channel.messages) {
    console.log(message);
    if (message.data.response == 'got comm open!') {
      const responseBuffer = new Uint8Array(message.buffers[0]);
      console.log(responseBuffer);
      for (let i = 0; i < buffer.length; ++i) {
        if (responseBuffer[i] != buffer[i]) {
          console.error('comm buffer different at ' + i);
          return;
        } else{
          console.log('buffers match!')
        }
      }
      // Close the channel once the expected message is received. This should
      // cause the messages iterator to complete and for the for-await loop to
      // end.
      channel.close();
    }
  }
  document.body.appendChild(document.createTextNode('done.'));
})()
''')

<IPython.core.display.Javascript object>

{'header': {'username': 'username', 'version': '5.0', 'msg_type': 'comm_open', 'msg_id': '9b6f740697b347a2ad52bcdb71e9c426', 'session': '6557d4958d874fd4d8563c96078d193a', 'date': datetime.datetime(2021, 4, 10, 20, 41, 17, 363985, tzinfo=tzlocal())}, 'msg_id': '9b6f740697b347a2ad52bcdb71e9c426', 'msg_type': 'comm_open', 'parent_header': {}, 'metadata': {}, 'content': {'target_name': 'comm_target', 'data': 'the data', 'comm_id': 'bbadbf48-ba21-498c-ad00-0251c2f76956'}, 'buffers': [<memory at 0x7fa327c23ae0>]}
